# Volúmenes finitos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from abc import ABC, abstractmethod
from sympy import *

In [2]:
class Finite_Volume(ABC):

    def __init__(self, Nx, xL, xR, Nt, tf, f):
        # Esquema a usar
        self.scheme = ""

        # Parametros grilla
        self.Nx = Nx
        self.Nt = Nt
        self.xL = xL
        self.xR = xR
        self.tf = tf

        # Pasos espacial y temporal
        self.dx = (xR - xL) / Nx
        self.dt = tf / (Nt + 1)

        # Arreglos
        self.x_mid = np.linspace(self.xL, self.xR, self.Nx+1) # puntos medios
        self.x = (self.x_mid[1:] + self.x_mid[:-1]) / 2 # puntos de la grilla

        self.t = np.linspace(0, self.tf, self.Nt)

        # Tiempo actual
        self.n = 0

        
    # Setea condición inicial
    def set_init_cond(self, U_0):
        
        # Guarda condición inicial
        U0 = U_0(self.x)
        
        # Inicializa solución
        self.U = np.zeros((self.Nt,) + U0.shape)
        
        self.U[0] = U0
        
        # Inicializa vector y función de flujo
        self.F = np.zeros(self.Nx+1)
        self.f = f
    
    # Obtiene derivada de f
    def get_f_prime(self):
        u = Symbol('u')
        y = self.f(u)
        f_prime = y.diff(u)
        self.f_prime = lambdify(u, f_prime)
    
    # Condición CFL
    def CFL_condition(self):
        self.get_f_prime()
        CFL = np.max(np.fabs(self.f_prime(self.U[self.n]))) * (self.dt/self.dx)

        if CFL > 1/2:
            print("Cuidado: No se cumple condición CFL")

    # Flujo esquema
    @abstractmethod
    def scheme_flux(self, u_left, u_right):
        pass
    
    # Flujo en bordes
    def border_flux(self):
        
        # Flujo en el borde izquierdo
        u_left = self.U[self.n, 0]
        u_right = self.U[self.n, 0]
        self.F[0] = self.scheme_flux(u_left, u_right)

        # Flujo en el borde derecho
        u_left = self.U[self.n, -1]
        u_right = self.U[self.n, -1]
        self.F[-1] = self.scheme_flux(u_left, u_right)

    
    # Calcula flujo
    def calculate_flux(self):
        for i in range(1, self.Nx):
            u_left = self.U[self.n, i-1]
            u_right = self.U[self.n, i]
            self.F[i] = self.scheme_flux(u_left, u_right)

        self.border_flux()

    # Actualiza solución
    def update(self):
        self.calculate_flux()

        # Actualiza solucion
        for i in range(self.Nx):
            self.U[self.n+1, i] = self.U[self.n, i] - (self.dt / self.dx) * (self.F[i+1] - self.F[i])
        self.n += 1
    
    # Obtiene animación
    def get_animation(self, title, path):
        fig = plt.figure()
        ax = plt.axes()
        plt.xlabel(r'$x$')
        plt.ylabel(r'$u(t, x)$')
        ax.set_title(title + " t=" + str(format(self.t[0], '.2f')) + " " + self.scheme)

        # prepare for animated lines
        line, = ax.plot(self.x, self.U[0], '-', color='r', linewidth=2, label="Simulación")

        def animate(n):
            line.set_ydata(self.U[n])

            ax.set_title(title + " t=" + str(format(self.t[n], '.2f')) + " " + self.scheme)
            return line,

        ani = animation.FuncAnimation(fig, animate, np.arange(0, self.Nt, 5),
                                    interval=70, blit=True)

        ani.save(path)
    
    # Obtiene animación con solución exacta
    def get_animation_exact(self, title, path, real_solution):
        fig = plt.figure()
        ax = plt.axes()
        plt.xlabel(r'$x$')
        plt.ylabel(r'$u(t, x)$')
        plt.title(title + " t=" + str(format(self.t[0], '.2f')) + " " + self.scheme)

        # prepare for animated lines
        line, = ax.plot(self.x, self.U[0], '-', color='r', linewidth=2, label="Simulación")
        exact_line, = ax.plot(self.x, real_solution(self.x, self.t[0]), color='b', linestyle='-', linewidth=2, label="Solución exacta")
        plt.legend()

        def animate(n):
            line.set_ydata(self.U[n])
            exact_line.set_ydata(real_solution(self.x, self.t[n]))

            plt.title(title + " t=" + str(format(self.t[n], '.2f')) + " " + self.scheme)
            return [line, exact_line] 

        ani = animation.FuncAnimation(fig, animate, np.arange(0, self.Nt, 5),
                                    interval=70, blit=True)

        ani.save(path)

## Esquema de Godunov

In [3]:
# Godunov
class Godunov_solver(Finite_Volume):

    def __init__(self, Nx, xL, xR, Nt, tf, f, flux):
        super().__init__(Nx, xL, xR, Nt, tf, f)
        self.flux = flux

        # Esquema
        self.scheme = "Godunov"

    # Flujo esquema
    def scheme_flux(self, u_left, u_right):
        return self.flux(u_left, u_right)

In [4]:
# Godunov con condiciones de borde periodicas
class Godunov_periodic_solver(Godunov_solver):

    def __init__(self, Nx, xL, xR, Nt, tf, f, flux):
        super().__init__(Nx, xL, xR, Nt, tf, f, flux)

    # Flujo en bordes
    def border_flux(self):
        # Flujo en el borde izquierdo
        u_left = self.U[self.n, -1]
        u_right = self.U[self.n, 0]
        self.F[0] = self.scheme_flux(u_left, u_right)

        # Flujo en el borde derecho
        self.F[-1] = self.scheme_flux(u_left, u_right)
    
    # Actualiza solución
    def update(self):
        self.calculate_flux()

        # Actualiza solucion
        for i in range(self.Nx):
            self.U[self.n+1, i] = self.U[self.n, i] - (self.dt / self.dx) * (self.F[i+1] - self.F[i])

        self.n += 1

In [5]:
# Godunov cuando f posee unico mínimo global
class Simplified_Godunov_solver(Finite_Volume):

    def __init__(self, Nx, xL, xR, Nt, tf, f, w):
        super().__init__(Nx, xL, xR, Nt, tf, f)

        # Punto mínimo
        self.w = w

        # Esquema
        self.scheme = "Godunov"

    # Flujo esquema
    def scheme_flux(self, u_left, u_right):
        u_l_positive = np.max([u_left, self.w])
        u_r_negative = np.min([u_right, self.w])

        return np.max([self.f(u_l_positive), self.f(u_r_negative)])

In [6]:
# Parametros grillas
Nx = 100
xL = -1
xR = 1
Nt = 400
tf = 2.5

# Flujo
def f(u):
    return u**2/2

# Condiciones iniciales
def U_0_1(x):
    return (x<0).astype(float)

# Solución real 1
# TODO: Modificar para que funcione con arreglos en x
def u_1(x, t):
    if x<t:
        return 1
    if x>0:
        return 0

def U_0_2(x):
    x_copy = x.copy()
    x_copy[x_copy<0] = -1
    x_copy[x_copy>=0] = 1
    return x_copy

In [7]:
# Solución con primera condición inicial
burgers_1 = Simplified_Godunov_solver(Nx, xL, xR, Nt, tf, f, 0)
burgers_1.set_init_cond(U_0_1)

for n in range(0,Nt-1):
    burgers_1.CFL_condition()
    burgers_1.update()

#burgers_1.get_animation("Burgers primera condición inicial", "volumenes_finitos/Primer_orden/Godunov/burgers_1.gif")

In [8]:
# Solucion con segunda condicion inicial
burgers_2 = Simplified_Godunov_solver(Nx, xL, xR, Nt, tf, f, 0)
burgers_2.set_init_cond(U_0_2)

for n in range(0,Nt-1):
    burgers_2.CFL_condition()
    burgers_2.update()

#burgers_2.get_animation("Burgers segunda condición inicial", "/work/volumenes_finitos/Primer_orden/Godunov/burgers_2.gif")

In [9]:
# Godunov con condiciones de borde periodicas
class Simplified_Godunov_periodic_solver(Simplified_Godunov_solver):

    def __init__(self, Nx, xL, xR, Nt, tf, f, w):
        super().__init__(Nx, xL, xR, Nt, tf, f, w)

    # Flujo en bordes
    def border_flux(self):
        # Flujo en el borde izquierdo
        u_left = self.U[self.n, -1]
        u_right = self.U[self.n, 0]
        self.F[0] = self.scheme_flux(u_left, u_right)

        # Flujo en el borde derecho
        self.F[-1] = self.scheme_flux(u_left, u_right)
    
    # Actualiza solución
    def update(self):
        self.calculate_flux()

        # Actualiza solucion
        for i in range(self.Nx):
            self.U[self.n+1, i] = self.U[self.n, i] - (self.dt / self.dx) * (self.F[i+1] - self.F[i])

        self.n += 1

In [10]:
def U_0_3(x):
    return np.cos(np.pi*x)

burgers_per = Simplified_Godunov_periodic_solver(Nx, xL, xR, Nt, tf, f, 0)
burgers_per.set_init_cond(U_0_3)

for n in range(0,Nt-1):
    burgers_per.CFL_condition()
    burgers_per.update()

#burgers_per.get_animation("Burgers periódico", "/work/volumenes_finitos/Primer_orden/Godunov/burgers_per.gif")

## Solvers linealizados

In [12]:
# Esquema linealizado
class Linearized_solver(Finite_Volume):

    def __init__(self, Nx, xL, xR, Nt, tf, f):
        super().__init__(Nx, xL, xR, Nt, tf, f)
        self.get_f_prime()

        # Esquema
        self.scheme = "Lineal"

    # Flujo esquema
    def scheme_flux(self, u_left, u_right):
        A = self.f_prime(u_left)
        if u_left != u_right:
            A = (self.f(u_right) - self.f(u_left))/(u_right - u_left)
        
        if A>= 0:
            return self.f(u_left)

        else:
            return self.f(u_right)
